<a href="https://colab.research.google.com/github/Erdgje95/Physics/blob/main/hfqpo_open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Open-Source HFQPO Model (Persistence + MHD)
This notebook simulates high-frequency QPOs using a first-principles model that combines:
- General relativity (GR) via spin-dependent ISCO
- MHD effects through dynamic magnetization boost

**Inputs:**
- Mass $M$ (in $M_\odot$)
- Spin $a$ (dimensionless Kerr spin)
- Plasma magnetization $B^2 / P_{\rm gas}$

The model predicts:
- HFQPO frequency without fine-tuning
- Frequency evolution across epochs or systems

In [1]:
# ---------- Cell 1 : imports & constants ----------
import numpy as np
import matplotlib.pyplot as plt

# Interactive widgets (Colab already has ipywidgets pre-installed)
import ipywidgets as widgets
from IPython.display import display, clear_output

# Physical constants (SI)
c   = 2.99792458e8        # m s⁻¹
G   = 6.67430e-11         # m³ kg⁻¹ s⁻²
M_s = 1.98847e30          # 1 M⊙ in kg



In [2]:
# ---------- Cell 2 : Kerr ISCO radius & base frequency ----------
def r_isco(M, a_dimless):
    """
    Kerr ISCO radius in metres (Bardeen+ 1972).
    M  : BH mass in kg
    a  : dimensionless spin  0 … 0.998
    """
    Z1 = 1 + (1 - a_dimless**2)**(1/3)*( (1+a_dimless)**(1/3) + (1-a_dimless)**(1/3) )
    Z2 = np.sqrt(3*a_dimless**2 + Z1**2)
    r_g = G*M / c**2                      # gravitational radius
    sign = np.sign(a_dimless)
    r_isco_over_rg = 3 + Z2 - sign*np.sqrt( (3-Z1)*(3+Z1 + 2*Z2) )
    return r_isco_over_rg * r_g           # metres

def f_vortex_n1(M_sol, a_dimless):
    """
    Quantised n=1 vortex frequency at ISCO in Hz (before MHD amplification).
    """
    M = M_sol * M_s
    r_i = r_isco(M, a_dimless)
    # orbital (Kepler) frequency at ISCO
    omega = np.sqrt(G*M / r_i**3)         # rad s⁻¹
    f_kepler = omega / (2*np.pi)          # Hz
    return f_kepler                       # ~ 50–1000 Hz range


In [3]:
# ---------- Cell 3 : helper dict for state-dependent slider ranges ----------
state_dict = {
    "thin / soft"        : (1e-4, 1e-2),
    "hard–intermediate"  : (1e-2, 1e+0),
    "MAD (plunging)"     : (1e+3, 1e+6),
}


In [4]:
# ---------- Cell 4 : widgets & interactive read-out ----------
# Sliders
mass_slider  = widgets.FloatSlider(value=10.0, min=2.0, max=20.0, step=0.1,
                                   description='Mass (M⊙)', continuous_update=False)
spin_slider  = widgets.FloatSlider(value=0.5, min=0.0, max=0.998, step=0.01,
                                   description='Spin a', continuous_update=False)
state_drop   = widgets.Dropdown(options=list(state_dict.keys()),
                                value="hard–intermediate",
                                description='Disk state')
logF_slider  = widgets.FloatSlider(value=-1.0, min=-2, max=0, step=0.02,
                                   description='log10 F_MHD', continuous_update=False)

# Text output
out = widgets.Output()

def update_F_range(change):
    lo, hi = np.log10(state_dict[state_drop.value])
    logF_slider.min, logF_slider.max = lo, hi
    # centre slider in the new range
    logF_slider.value = 0.5*(lo+hi)
state_drop.observe(update_F_range, names='value')

def refresh(_=None):
    with out:
        clear_output(wait=True)
        M   = mass_slider.value
        a   = spin_slider.value
        F   = 10**logF_slider.value
        f0  = f_vortex_n1(M, a)
        fHF = F * f0
        print(f"Base vortex frequency  : {f0:7.1f}  Hz")
        print(f"MHD amplification F    : {F:7.3g}")
        print(f"\u2192  Predicted HFQPO  : {fHF:7.1f}  Hz")

for w in (mass_slider, spin_slider, state_drop, logF_slider):
    w.observe(refresh, names='value')

display(widgets.VBox([mass_slider, spin_slider, state_drop, logF_slider, out]))
refresh()


In [5]:
# ---------- Cell 5 : buttons to jump to published M & a ----------
preset = {
    "GRO J1655–40"  : (6.3, 0.70),
    "XTE J1550–564" : (9.1, 0.34),
    "GRS 1915+105"  : (12.4, 0.98),
}

def make_button(name, Ma):
    b = widgets.Button(description=name, layout=widgets.Layout(width='160px'))
    def on_click(_):
        mass_slider.value, spin_slider.value = Ma
    b.on_click(on_click)
    return b

buttons = widgets.HBox([make_button(n,v) for n,v in preset.items()])
display(buttons)


In [6]:
# ───── Cell 6 (corrected): Δ-model & c_s sweep ─────
import numpy as np
import pandas as pd
from scipy.constants import h, pi

# 1) Effective boson mass (CFL pairing): m_* ≈ 200 MeV/c² in kg
m_star = 200e6 * 1.60218e-19 / c**2

# 2) Three Δ(r) prescriptions:
def delta_powerlaw(rho, Δ0=50e6*1.60218e-19, rho0=3e18):
    return Δ0 * (rho/rho0)**(1/3)

def delta_schafer(rho):
    μ = (3*pi**2*rho/0.16)**(1/3) * 0.938e9
    return (30e6*1.60218e-19) * np.exp(-200/μ)

def delta_alford(rho):
    return delta_powerlaw(rho, Δ0=80e6*1.60218e-19, rho0=5e18)

Δ_models = {
    'PowerLaw(50 MeV)': delta_powerlaw,
    'Schäfer 2003':       delta_schafer,
    'Alford 2008':        delta_alford,
}

# 3) Analytic density at ISCO (ρ ∝ r^−3.3 → set ρ_ISCO = 3e18 kg/m³)
rho_isco = 3e18

# 4) Assume Bφ = 1e7 G (Gauss)
Bphi = 1e7

# 5) Quantum of circulation
κ = h/(m_star * c)

# 6) Extended compute_f_env (adds Δ, ξ, N)
def compute_f_env_ext(M_sol, a_dim, Δ_func, cs_frac):
    M = M_sol * M_s
    # r_ISCO
    rI = r_isco(M, a_dim)
    # Δ and coherence length
    ΔI = Δ_func(rho_isco)
    vF = cs_frac * c/np.sqrt(3)
    ξI = (h/(2*pi)) * vF / ΔI
    # bundle occupation N
    rho_s = rho_isco
    N  = (Bphi*1e-4)**2 / (4*pi * κ * rho_s)
    # envelope frequency
    fE = (cs_frac * c) / (4 * rI)
    return rI, ΔI, ξI, N, fE

# 7) Sweep over sources, Δ-models, and c_s fractions
rows = []
for src,(M,a) in preset.items():
    for name, Δf in Δ_models.items():
        for csf in [0.5, 0.6, 0.7]:
            rI, ΔI, ξI, N, fE = compute_f_env_ext(M, a, Δf, csf)
            rows.append([src, name, csf, rI, ΔI, ξI, N, fE])

df_sweep = pd.DataFrame(rows, columns=[
    'Source','Δ_model','c_s_frac','r_ISCO(m)',
    'Δ_ISCO(J)','ξ_ISCO(m)','N','f_env(Hz)'
])
df_sweep


,Source,Δ_model,c_s_frac,r_ISCO(m),Δ_ISCO(J),ξ_ISCO(m),N,f_env(Hz)
0,GRO J1655–40,PowerLaw(50 MeV),0.5,31566.338809,8.010900e-12,1.139265e-15,4.278913,1187.152475
1,GRO J1655–40,PowerLaw(50 MeV),0.6,31566.338809,8.010900e-12,1.367119e-15,4.278913,1424.582970
2,GRO J1655–40,PowerLaw(50 MeV),0.7,31566.338809,8.010900e-12,1.594972e-15,4.278913,1662.013465
3,GRO J1655–40,Schäfer 2003,0.5,31566.338809,4.806540e-12,1.898776e-15,4.278913,1187.152475
4,GRO J1655–40,Schäfer 2003,0.6,31566.338809,4.806540e-12,2.278531e-15,4.278913,1424.582970
5,GRO J1655–40,Schäfer 2003,0.7,31566.338809,4.806540e-12,2.658286e-15,4.278913,1662.013465
6,GRO J1655–40,Alford 2008,0.5,31566.338809,1.081065e-11,8.442179e-16,4.278913,1187.152475
7,GRO J1655–40,Alford 2008,0.6,31566.338809,1.081065e-11,1.013061e-15,4.278913,1424.582970
8,GRO J1655–40,Alford 2008,0.7,31566.338809,1.081065e-11,1.181905e-15,4.278913,1662.013465
9,XTE J1550–564,PowerLaw(50 MeV),0.5,64967.073338,8.010900e-12,1.139265e-15,4.278913,576.816152


In [7]:
# ───── Cell 7 (fixed): Boundary-layer damping & Q ─────
import numpy as np
import pandas as pd
from scipy.constants import pi

rows = []
for _, row in df_sweep.iterrows():
    # pick up the f_env value by its exact column name
    f_env = row['f_env(Hz)']
    ω_env = 2 * pi * f_env
    for α in [0.01, 0.05, 0.1]:
        Q = ω_env / (2 * (α * ω_env))
        rows.append([
            row['Source'],
            row['Δ_model'],
            row['c_s_frac'],
            α,
            Q
        ])

df_Q = pd.DataFrame(rows, columns=[
    'Source', 'Δ_model', 'c_s_frac', 'alpha_damp', 'Q_factor'
])
df_Q


,Source,Δ_model,c_s_frac,alpha_damp,Q_factor
0,GRO J1655–40,PowerLaw(50 MeV),0.5,0.01,50.0
1,GRO J1655–40,PowerLaw(50 MeV),0.5,0.05,10.0
2,GRO J1655–40,PowerLaw(50 MeV),0.5,0.10,5.0
3,GRO J1655–40,PowerLaw(50 MeV),0.6,0.01,50.0
4,GRO J1655–40,PowerLaw(50 MeV),0.6,0.05,10.0
...,...,...,...,...,...
76,GRS 1915+105,Alford 2008,0.6,0.05,10.0
77,GRS 1915+105,Alford 2008,0.6,0.10,5.0
78,GRS 1915+105,Alford 2008,0.7,0.01,50.0
79,GRS 1915+105,Alford 2008,0.7,0.05,10.0


In [8]:
# ───── Cell 8: MRI‐turbulence jitter test ─────
import numpy as np

def jitter_test(M,a,cs_frac=0.6,τ=0.1,rms=0.2,steps=5000):
    # base f_env
    _,_,_,_,f0 = compute_f_env_ext(M,a,delta_powerlaw,cs_frac)
    dt,α = 1/1000, np.exp(-1/1000/τ)
    B=1.0; fs=[]
    for i in range(steps):
        B = α*B + np.sqrt(1-α**2)*rms*np.random.randn()
        fs.append(f0)
    return np.std(fs)/np.mean(fs)

for src,(M,a) in preset.items():
    rel = jitter_test(M,a)
    print(f"{src}: relative f_env jitter = {rel:.1e}")


GRO J1655–40: relative f_env jitter = 1.6e-16
XTE J1550–564: relative f_env jitter = 1.6e-16
GRS 1915+105: relative f_env jitter = 3.0e-16
